In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat May 24 17:25:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   60C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## No GPU? No problem!  
Just activate the GPU in Colab (ask a mentor if you need help), then start from here.

In [ ]:
!pip install transformers[torch] evaluate accelerate torchaudio librosa
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
!pip install hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 103.9 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, Audio
import re
from huggingface_hub import login


token = ""

login(token=token)

dataset_name = "atlasia/DODa-audio-dataset"
raw_dataset = load_dataset(dataset_name, split="train[:30%]", token=token) 
print("Raw dataset loaded:", raw_dataset)

README.md:   0%|          | 0.00/5.36k [00:00<?, ?B/s]

data/train-00000-of-00005.parquet:   0%|          | 0.00/333M [00:00<?, ?B/s]

data/train-00001-of-00005.parquet:   0%|          | 0.00/279M [00:00<?, ?B/s]

data/train-00002-of-00005.parquet:   0%|          | 0.00/237M [00:00<?, ?B/s]

data/train-00003-of-00005.parquet:   0%|          | 0.00/226M [00:00<?, ?B/s]

data/train-00004-of-00005.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12743 [00:00<?, ? examples/s]

Raw dataset loaded: Dataset({
    features: ['audio', 'darija_Latn', 'darija_Arab_new', 'english', 'darija_Arab_old'],
    num_rows: 3823
})


In [ ]:

def extract_all_chars(batch):
    transcription_key = "darija_Arab_new"
    all_text = " ".join(t for t in batch[transcription_key] if t is not None)
    vocab = list(set(all_text.lower())) 
    return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:

vocabs = raw_dataset.map(extract_all_chars, batched=True, batch_size=8, keep_in_memory=True, remove_columns=raw_dataset.column_names)
vocab_list = list(set(c for vocab_item in vocabs["vocab"] for c in vocab_item))

Map:   0%|          | 0/3823 [00:00<?, ? examples/s]

In [ ]:
vocab_list

[':',
 'ء',
 'ا',
 'ى',
 'ض',
 '؟',
 '-',
 '1',
 'ب',
 'آ',
 'ؤ',
 'و',
 'ن',
 'ل',
 'ث',
 ',',
 '،',
 'ج',
 'م',
 'ي',
 'د',
 'إ',
 'ڤ',
 'س',
 '!',
 'ئ',
 'ق',
 'ص',
 '.',
 '9',
 'ك',
 'غ',
 'ح',
 'ه',
 'ز',
 'ة',
 'ش',
 'خ',
 ' ',
 'ف',
 'ّ',
 'پ',
 'ر',
 'ُ',
 'ظ',
 'ڭ',
 'ت',
 '6',
 'ط',
 'ع',
 'ذ',
 'أ',
 '?']

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    if batch["darija_Arab_new"] == None:
      batch["text"] = "خاوي"
    else:
      batch["text"] = re.sub(chars_to_ignore_regex, '', batch["darija_Arab_new"]).lower() + " "
    return batch

def remove_digits(batch):
    if batch["darija_Arab_new"] == None:
      batch["text"] = "خاوي"
    else:
      batch["text"] = re.sub(r'\d+', '', batch["text"])
    return batch

def normalize(batch):
    if batch["darija_Arab_new"] == None:
      batch["text"] = "خاوي"
    else:
      hamza_pattern = re.compile(r'[ءأإؤئيآ]')
      batch["text"] = re.sub(hamza_pattern, 'ء', batch["text"])
    return batch


In [ ]:
raw_dataset = raw_dataset.map(remove_special_characters)
raw_dataset = raw_dataset.map(remove_digits)
raw_dataset = raw_dataset.map(normalize)

Map:   0%|          | 0/3823 [00:00<?, ? examples/s]

In [ ]:
raw_dataset

Dataset({
    features: ['audio', 'darija_Latn', 'darija_Arab_new', 'english', 'darija_Arab_old', 'text'],
    num_rows: 3823
})

In [ ]:
def extract_all_chars(batch):
    transcription_key = "text"
    all_text = " ".join(t for t in batch[transcription_key] if t is not None)
    vocab = list(set(all_text.lower())) 
    return {"vocab": [vocab], "all_text": [all_text]}

vocabs = raw_dataset.map(extract_all_chars, batched=True, batch_size=8, keep_in_memory=True, remove_columns=raw_dataset.column_names)
vocab_list = list(set(c for vocab_item in vocabs["vocab"] for c in vocab_item))

Map:   0%|          | 0/3823 [00:00<?, ? examples/s]

In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

49

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
raw_dataset = raw_dataset.map(prepare_dataset)

Map:   0%|          | 0/3823 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [ ]:
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:

        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        labels_batch = self.processor.pad(
            labels=label_features,
            padding=self.padding,
            return_tensors="pt",
        )

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels
        return batch

if processor:
    data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
    print("Data collator defined.")
else:
    data_collator = None
    print("Some Problem is there, call Mentor.")


Data collator defined.


In [ ]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.4 MB/s eta 0:00:00


In [ ]:
import evaluate

wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred_str = processor.batch_decode(pred_ids)
    label_ids_cleaned = []
    for label_seq in pred.label_ids:
        label_ids_cleaned.append([token_id for token_id in label_seq if token_id != -100 and token_id != processor.tokenizer.pad_token_id])
    label_str = processor.batch_decode(label_ids_cleaned, group_tokens=False) # group_tokens=False for char-level

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

print("Metrics functions defined.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Metrics functions defined.


In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "jonatasgrosman/wav2vec2-large-xlsr-53-arabic", 
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer) 
)
model.freeze_feature_encoder()
print("Pre-trained model loaded.")

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Pre-trained model loaded.


In [ ]:
from transformers import TrainingArguments

output_dir = "./wav2vec2-finetuned-demo"


training_args = TrainingArguments(
    output_dir=output_dir,
    group_by_length=True, 
    per_device_train_batch_size=2, 
    per_device_eval_batch_size=2,
    eval_strategy="steps",
    num_train_epochs=3,
    save_steps=50, 
    eval_steps=50, 
    logging_steps=50, 
    learning_rate=3e-4, 
    report_to="none",
)
print("Training arguments defined.")

Training arguments defined.


In [ ]:
from transformers import Trainer
import numpy as np 
import dataclasses 
from typing import Dict, List, Optional, Union 

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=raw_dataset,
    eval_dataset=raw_dataset,  
    tokenizer=processor.feature_extractor, 
)

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

<ipython-input-26-6824c13d82ad>:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()
print("Fine-tuning completed.")
model_save_path = f"{output_dir}/final_model"
processor.save_pretrained(model_save_path)
trainer.save_model(model_save_path)
print(f"Final model and processor saved to {model_save_path}")

Step,Training Loss,Validation Loss,Wer
50,3.828200,3.563410,1.000519
100,3.137300,3.662459,0.999793


KeyboardInterrupt: 

In [ ]:
print("Fine-tuning completed.")
model_save_path = f"{output_dir}/final_model"
processor.save_pretrained(model_save_path)
trainer.save_model(model_save_path)
print(f"Final model and processor saved to {model_save_path}")

In [ ]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from datasets import Dataset, Audio
from jiwer import wer
import os

processor = Wav2Vec2Processor.from_pretrained("boumehdi/wav2vec2-large-xlsr-moroccan-darija")
model = Wav2Vec2ForCTC.from_pretrained("boumehdi/wav2vec2-large-xlsr-moroccan-darija").cuda()

def map_to_result(batch):
    with torch.no_grad(), torch.cuda.amp.autocast():
        input_values = processor(
            batch["audio"]["array"],
            sampling_rate=batch["audio"]["sampling_rate"],
            return_tensors="pt"
        ).input_values.to('cuda')

        logits = model(input_values).logits
        pred_ids = torch.argmax(logits, dim=-1)

        batch["pred_str"] = processor.batch_decode(pred_ids, skip_special_tokens=True)[0]

    return batch

    return batch

In [ ]:

def load_audio_files(audio_paths):
    data = {"audio": audio_paths}
    dataset = Dataset.from_dict(data).cast_column("audio", Audio(sampling_rate=16000))
    return dataset

def evaluate_samples(audio_folder, references):

    audio_files = [os.path.join(audio_folder, f"{i}.wav") for i in range(3)]

    dataset = load_audio_files(audio_files)

    results = dataset.map(map_to_result)

    wers = []
    for i, ref in enumerate(references):
        hyp = results[i]["pred_str"]
        error = wer(ref, hyp)
        wers.append((i, error, ref, hyp))

    return wers